In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from scipy.special import softmax
import multiprocessing as mp
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from torch.nn import functional as F
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

# Installing ProdLDA
**Restart notbook after the installation!!**

In [2]:
!git clone https://github.com/estebandito22/PyTorchAVITM

fatal: destination path 'PyTorchAVITM' already exists and is not an empty directory.


# 1. Creation of synthetic corpus

We consider a scenario with n parties, each of them as an associated corpus.
To generate the corpus associated with each of the parties, we consider a common beta distribution (word-topic distribution), but we freeze different topics/ assign different asymmetric Dirichlet priors favoring different topics at the time of generating the document that composes each party's corpus.

## 1.1. Function for permuting the Dirichlet prior at each node

In [3]:
def rotateArray(arr, n, d):
    temp = []
    i = 0
    while (i < d):
        temp.append(arr[i])
        i = i + 1
    i = 0
    while (d < n):
        arr[i] = arr[d]
        i = i + 1
        d = d + 1
    arr[:] = arr[: i] + temp
    return arr

## 1.2. Topic modeling and node settings

In [4]:
# Topic modeling settings
vocab_size = 5000
n_topics = 50
beta = 1e-2
alpha = 1/n_topics
n_docs = 1000
nwords = (150, 250) #Min and max lengths of the documents

# Nodes settings
n_nodes = 5
frozen_topics = 5
prior_frozen = frozen_topics * [alpha]
own_topics = int((n_topics-frozen_topics)/n_nodes)
prior_nofrozen = own_topics * [alpha] + (n_topics-frozen_topics-own_topics) * [alpha/10000]
#print(prior_frozen + prior_nofrozen)

## 1.3. Topics generation (common for all nodes)

In [5]:
topic_vectors = np.random.dirichlet(vocab_size*[beta], n_topics)
print('Probabilidades ordenadas para el primer vector de tópicos:')
print(np.sort(topic_vectors[0])[::-1])
print(topic_vectors.shape)

Probabilidades ordenadas para el primer vector de tópicos:
[0.06343621 0.05713653 0.05458389 ... 0.         0.         0.        ]
(50, 5000)


In [6]:
#Here we compare alignment of the topic_vector matrix with itself and with another randomly generated matrix
print('Tópicos (equivalentes) identificados correctamente (true):', np.sum(np.max(np.sqrt(topic_vectors).dot(np.sqrt(topic_vectors.T)), axis=0)))
topic_vectors2 = np.random.dirichlet(vocab_size*[beta], n_topics)
print('Tópicos (equivalentes) identificados correctamente (random):', np.sum(np.max(np.sqrt(topic_vectors2).dot(np.sqrt(topic_vectors.T)), axis=0)))

Tópicos (equivalentes) identificados correctamente (true): 50.00000000000006
Tópicos (equivalentes) identificados correctamente (random): 3.660410001730459


## 1.4. Generation of document topic proportions and documents for each node


In [7]:
# Step 2 - generation of document topic proportions
doc_topics_all = []
for i in np.arange(n_nodes):
    doc_topics = np.random.dirichlet(prior_frozen + prior_nofrozen, n_docs)
    prior_nofrozen = rotateArray(prior_nofrozen, len(prior_nofrozen), own_topics)
    doc_topics_all.append(doc_topics)

In [8]:
# Step 3 - Document generation
documents_all = []
z_all = []

for i in np.arange(n_nodes):
    documents = [] # Document words
    #z = [] # Assignments
    for docid in np.arange(n_docs):
        doc_len = np.random.randint(low=nwords[0], high=nwords[1])
        this_doc_words = []
        #this_doc_assigns = []
        for wd_idx in np.arange(doc_len):
            tpc = np.nonzero(np.random.multinomial(1, doc_topics_all[i][docid]))[0][0]
            #this_doc_assigns.append(tpc)
            word = np.nonzero(np.random.multinomial(1, topic_vectors[tpc]))[0][0]
            this_doc_words.append('wd'+str(word))
        #z.append(this_doc_assigns)
        documents.append(this_doc_words)
    documents_all.append(documents)
    #z_all.append(z)

In [9]:
np.savez('synthetic_10000_beta_1.npz', n_nodes = n_nodes, vocab_size=vocab_size, n_topics=n_topics, frozen_topics = frozen_topics, beta=beta, alpha=alpha,
        n_docs=n_docs, nwords=nwords, topic_vectors=topic_vectors, doc_topics=doc_topics_all,
        documents=documents_all, z=z_all)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [10]:
doc_topics_all_gt = doc_topics_all

# 2. Auxiliary functions

In [11]:
cd /content/PyTorchAVITM/pytorchavitm/datasets

/content/PyTorchAVITM/pytorchavitm/datasets


In [12]:
from bow import BOWDataset

In [13]:
cd /content/PyTorchAVITM

/content/PyTorchAVITM


In [14]:
from pytorchavitm import AVITM

In [15]:
def prepare_dataset(docs_train, docs_val):

  # Get train-test split for each node and convert into AVITM format
  cv = CountVectorizer(input='content', lowercase=True, stop_words='english',
                       max_df=0.99, min_df=0.01, binary=False)

  # Generate AVITM dataset for correspinding node
  docs_train_conv = [" ".join(docs_train[i]) for i in np.arange(len(docs_train))]
  train_bow = cv.fit_transform(docs_train_conv)
  train_bow = train_bow.toarray()
  idx2token = cv.get_feature_names()
  input_size = len(idx2token)
  id2token = {k: v for k, v in zip(range(0, len(idx2token)), idx2token)}
  train_data = BOWDataset(train_bow, idx2token)

  docs_val_conv = [" ".join(docs_val[i]) for i in np.arange(len(docs_val))]
  val_bow = cv.transform(docs_val_conv)
  val_bow = val_bow.toarray()
  val_data = BOWDataset(val_bow, idx2token)

  return train_data, val_data, input_size, id2token

In [16]:
def train_avitm(docs_train, input_size_):
  """Trains an AVITM model with ProdLDA."""   

  avitm = AVITM(input_size=input_size_, n_components=n_topics, model_type='prodLDA',
                hidden_sizes=(100, 100), activation='softplus', dropout=0.2,
                learn_priors=True, batch_size=64, lr=2e-3, momentum=0.99,
                solver='adam', num_epochs=100, reduce_on_plateau=True)
  
  avitm.fit(docs_train)

  return avitm

In [28]:
def eval_avitm(val_data, avtim, k):
  avtim.model.eval()
  
  loader = DataLoader(
      val_data, batch_size=avtim.batch_size, shuffle=False,
      num_workers=mp.cpu_count())

  preds = []

  with torch.no_grad():
    for batch_samples in loader:
      # batch_size x vocab_size
      X = batch_samples['X']

      if avtim.USE_CUDA:
        X = X.cuda()

      # forward pass
      avtim.model.zero_grad()
      values = avtim.model(X)
      word_dists = values[-1]

      _, indices = torch.sort(word_dists, dim=1)
      preds += [indices[:, :k]]

    preds = torch.cat(preds, dim=0)

  n_samples = 20
  pbar = tqdm(n_samples, position=0, leave=True)

  pred_thetas = []
  for sample_index in range(n_samples):
    with torch.no_grad():
      collect_theta = []

      for batch_samples in loader:
        X = batch_samples['X']

        if avtim.USE_CUDA:
          X = X.cuda()

        # forward pass
        avtim.model.zero_grad()
        
        with torch.no_grad():
          posterior_mu, posterior_log_sigma = avtim.model.inf_net(X)

          # Generate samples from theta
          theta = F.softmax(
                  avtim.model.reparameterize(posterior_mu, posterior_log_sigma), dim=1)
          theta = avtim.model.drop_theta(theta)

        collect_theta.extend(theta.cpu().numpy().tolist())

      pbar.update(1)
      pbar.set_description("Sampling: [{}/{}]".format(sample_index + 1, n_samples))

      pred_thetas.append(np.array(collect_theta))
          
  pbar.close()
  pred_thetas = np.sum(pred_thetas, axis=0) / n_samples

  return preds,pred_thetas

In [18]:
def get_doc_topic_distribution(avitm, dataset, n_samples=20):
    """Given a trained AVITM model, it gets its associated document-topic distribution.

    Args:
        * n_samples (int, optional): Defaults to 20.

    Returns:
        * ndarray : Document-topics distribution
    """     
    avitm.model.eval()

    loader = DataLoader(
            avitm.train_data, batch_size=avitm.batch_size, shuffle=True,
            num_workers=mp.cpu_count())

    pbar = tqdm(n_samples, position=0, leave=True)

    final_thetas = []
    for sample_index in range(n_samples):
        with torch.no_grad():
            collect_theta = []

            for batch_samples in loader:
                X = batch_samples['X']

                if avitm.USE_CUDA:
                  X = X.cuda()

                # forward pass
                avitm.model.zero_grad()
                
                with torch.no_grad():
                  posterior_mu, posterior_log_sigma = avitm.model.inf_net(X)

                  # Generate samples from theta
                  theta = F.softmax(
                          avitm.model.reparameterize(posterior_mu, posterior_log_sigma), dim=1)
                  theta = avitm.model.drop_theta(theta)

                collect_theta.extend(theta.cpu().numpy().tolist())

            pbar.update(1)
            pbar.set_description("Sampling: [{}/{}]".format(sample_index + 1, n_samples))

            final_thetas.append(np.array(collect_theta))
    pbar.close()
    return np.sum(final_thetas, axis=0) / n_samples

In [19]:
def get_topic_word_distribution(avtim_model):
  """Given a trained AVITM model, it gets its associated topic-word distribution.

    Args:
        * avtim_model (AVITM): Trained AVITM model.

    Returns:
        * ndarray : topic-word distribution
    """     
  topic_word_matrix = avtim_model.best_components.cpu().detach().numpy()
  wd = softmax(topic_word_matrix, axis=1)
  return normalize(wd,axis=1,norm='l1')

In [20]:
def convert_topic_word_to_init_size(vocab_size, model, ntopics,
                                    id2token, all_words):
    """It converts the topic-word distribution matrix obtained from the
    training of a model into a matrix with the dimensions of the original
    topic-word distribution, assigning zeros to those words that are not
    present in the corpus. 
    It is only of use in case we are training a model over a synthetic dataset,
    so as to later compare the performance of the attained model in what regards
    to the similarity between the original and the trained model.

    Args:
        * vocab_size (int):       Size of the synethic'data vocabulary.
        * model (AVITM):          Model whose topic-word matrix is being transformed.
        * ntopics (int):          Number of topics of the trained model.
        * id2token (List[tuple]): Mappings with the content of the document-term matrix.
        * all_words (List[str]):  List of all the words of the vocabulary of size vocab_size.

    Returns:
        * ndarray: Normalized transormed topic-word distribution.
    """
    w_t_distrib = np.zeros((ntopics, vocab_size), dtype=np.float64)
    wd = get_topic_word_distribution(model)
    for i in np.arange(ntopics):
        for idx, word in id2token.items():
            for j in np.arange(len(all_words)):
                if all_words[j] == word:
                    w_t_distrib[i,j] = wd[i][idx]
                    break
    return w_t_distrib
    

# 2. Generate datasets

## 2.1. Train and validations datasets for the node 0

In [21]:
# Get documents and document-topic proportions associated to the node 0
cut = int(len(documents_all[0])/4)
cut = 3*cut
docs_node_train = documents_all[0][0:cut]
docs_node_val = documents_all[0][cut:]
doc_topics_train_0 = doc_topics_all[0][0:cut,:]
doc_topics_val_0 = doc_topics_all[0][cut:,:]

# Prepare node 0's dataset in AVITM format
documents_train_0, documents_val_0, input_size_0, id2token_0 = prepare_dataset(docs_node_train, docs_node_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## 2.2. Train and validation datasets for the centralized model

In [22]:
# Get documents and document-topic proportions for the centralized model

# Divide between train and val docs for each node 
docs_centr_train = []
docs_centr_val = []
doc_topics_train_centr = []
doc_topics_val_centr = []
for node in range(n_nodes):
  docs_centr_train.append(documents_all[node][0:cut])
  docs_centr_val.append(documents_all[node][cut:])
  doc_topics_train_centr.append(doc_topics_all[node][0:cut,:])
  doc_topics_val_centr.append(doc_topics_all[node][cut:,:])

# Collapse all centralized documents into one list for training and another one for validation.
# Same applyes for the doc-tops, but they are concatenated into arrays
documents_centr_train = [*docs_centr_train[0], *docs_centr_train[1], *docs_centr_train[2], *docs_centr_train[3], *docs_centr_train[4]]
documents_centr_val = [*docs_centr_val[0], *docs_centr_val[1], *docs_centr_val[2], *docs_centr_val[3], *docs_centr_val[4]]
doc_topics_train_centr = np.concatenate((doc_topics_train_centr[0], doc_topics_train_centr[1], doc_topics_train_centr[2], doc_topics_train_centr[3], doc_topics_train_centr[4]), axis=0)
doc_topics_val_centr = np.concatenate((doc_topics_val_centr[0], doc_topics_val_centr[1], doc_topics_val_centr[2], doc_topics_val_centr[3], doc_topics_val_centr[4]), axis=0)

# Prepare centralized dataset in AVITM format
documents_train_centr, documents_val_centr, input_size_centr, id2token_centr = prepare_dataset(documents_centr_train, documents_centr_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 3. Centralized ProdLDA model

In [23]:
avitm_centr = train_avitm(documents_train_centr, input_size_centr)

Settings: 
               N Components: 50
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.98
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: True
               Save Dir: None
Epoch: [1/100]	Samples: [3750/375000]	Train Loss: 1528.4835270833332	Time: 0:00:00.995303
Epoch: [2/100]	Samples: [7500/375000]	Train Loss: 1396.4731166666666	Time: 0:00:00.977431
Epoch: [3/100]	Samples: [11250/375000]	Train Loss: 1347.6133635416666	Time: 0:00:00.968912
Epoch: [4/100]	Samples: [15000/375000]	Train Loss: 1308.5852833333333	Time: 0:00:00.968127
Epoch: [5/100]	Samples: [18750/375000]	Train Loss: 1283.800190625	Time: 0:00:00.979791
Epoch: [6/100]	Samples: [22500/375000]	Train Loss: 1262.3216604166666	Time: 0:00:00.936936
Epoch: [7/100]	Samples: [26250

In [24]:
doc_topic_centr = get_doc_topic_distribution(avitm_centr, documents_train_centr, n_samples=5) # get all the topic predictions
print(doc_topic_centr.shape)

Sampling: [5/5]: : 5it [00:02,  2.38it/s]

(3750, 50)


In [25]:
all_words = ['wd'+str(word) for word in np.arange(vocab_size+1) if word > 0]

In [26]:
word_topic_centr = convert_topic_word_to_init_size(vocab_size, avitm_centr, n_topics, id2token_centr, all_words)
word_topic_centr.shape

(50, 5000)

In [29]:
preds_centr, pred_thetas_centr = eval_avitm(documents_val_centr, avitm_centr, 50)
print(pred_thetas_centr)

Sampling: [20/20]: : 20it [00:04,  4.39it/s]

[[0.01147759 0.01327403 0.01104251 ... 0.00935853 0.00835784 0.01697652]
 [0.0075569  0.01239275 0.0103828  ... 0.0553876  0.01547357 0.01014435]
 [0.00678008 0.00623024 0.00678342 ... 0.0130512  0.01041772 0.00788356]
 ...
 [0.01820842 0.00757589 0.0117983  ... 0.00814694 0.01093852 0.00708112]
 [0.00453051 0.00746715 0.0038398  ... 0.00917913 0.01110446 0.00441258]
 [0.01358224 0.00736376 0.01328202 ... 0.01115559 0.00442354 0.00213456]]


# 4. Node 0

In [30]:
avitm = train_avitm(documents_train_0, input_size_0)

Settings: 
               N Components: 50
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.98
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: True
               Save Dir: None
Epoch: [1/100]	Samples: [750/75000]	Train Loss: 1528.421875	Time: 0:00:00.208241
Epoch: [2/100]	Samples: [1500/75000]	Train Loss: 1464.0140520833334	Time: 0:00:00.230494
Epoch: [3/100]	Samples: [2250/75000]	Train Loss: 1403.3408229166666	Time: 0:00:00.236690
Epoch: [4/100]	Samples: [3000/75000]	Train Loss: 1356.2418125	Time: 0:00:00.218632
Epoch: [5/100]	Samples: [3750/75000]	Train Loss: 1322.8078645833334	Time: 0:00:00.207862
Epoch: [6/100]	Samples: [4500/75000]	Train Loss: 1291.7146770833333	Time: 0:00:00.227724
Epoch: [7/100]	Samples: [5250/75000]	Train Loss: 1

### Document-topic distributions

In [31]:
doc_topic = get_doc_topic_distribution(avitm, documents_train_0, n_samples=5) # get all the topic predictions
print("Document-topic distribution node 0")
print(np.array(doc_topic).shape)

Sampling: [5/5]: : 5it [00:00,  7.11it/s]

Document-topic distribution node 0
(750, 50)


### Word-topic distributions 

In [32]:
word_topic = convert_topic_word_to_init_size(vocab_size, avitm, n_topics, id2token_0, all_words)
word_topic.shape
print(word_topic)
sum(word_topic[0,:])

[[0.00059334 0.         0.         ... 0.         0.00065115 0.        ]
 [0.00063145 0.         0.         ... 0.         0.00063369 0.        ]
 [0.00059186 0.         0.         ... 0.         0.00063848 0.        ]
 ...
 [0.00083595 0.         0.         ... 0.         0.00049691 0.        ]
 [0.00056034 0.         0.         ... 0.         0.00064591 0.        ]
 [0.00055954 0.         0.         ... 0.         0.00065185 0.        ]]


1.0000000567233656

In [33]:
preds_node0, pred_thetas = eval_avitm(documents_val_0, avitm, 50)
print(preds_node0)

Sampling: [20/20]: : 20it [00:01, 10.13it/s]

tensor([[ 732, 1054, 1323,  ..., 1121,  677, 1607],
        [1192,  360,  614,  ..., 1611,  321, 1431],
        [ 914,  142,  515,  ...,  233, 1467,  803],
        ...,
        [ 273,  271,  542,  ...,  325, 1100,   43],
        [ 732, 1054,  854,  ...,  383,  978,  867],
        [1535,  519, 1425,  ...,  508,  974, 1143]])


# 4. Evaluation

In [34]:
doc_topic_centr_node_0_train = doc_topic_centr[0:cut,:]
doc_topic_centr_node_0_val = doc_topic_centr[cut:n_docs,:]

### Doc-topics

In [35]:
# doc_topics_all[0]
sim_mat_theoretical_train = np.sqrt(doc_topics_train_0).dot(np.sqrt(doc_topics_train_0.T))
sim_mat_actual_train = np.sqrt(doc_topic).dot(np.sqrt(doc_topic.T))
print('Difference in evaluation (train) of doc similarity (node 0):', np.sum(np.abs(sim_mat_theoretical_train - sim_mat_actual_train))/len(doc_topics_train_0))

sim_mat_theoretical_val = np.sqrt(doc_topics_val_0).dot(np.sqrt(doc_topics_val_0.T))
sim_mat_actual_val = np.sqrt(pred_thetas).dot(np.sqrt(pred_thetas.T))
print('Difference in evaluation (val) of doc similarity (node 0):', np.sum(np.abs(sim_mat_theoretical_val - sim_mat_actual_val))/len(doc_topics_val_0))

sim_mat_actual_centr_train = np.sqrt(doc_topic_centr_node_0_train).dot(np.sqrt(doc_topic_centr_node_0_train.T))
print('Difference in evaluation (train) of doc similarity (centr):', np.sum(np.abs(sim_mat_theoretical_train - sim_mat_actual_centr_train))/len(doc_topics_train_0))

sim_mat_actual_centr_val= np.sqrt(doc_topic_centr_node_0_val).dot(np.sqrt(doc_topic_centr_node_0_val.T))
print('Difference in evaluation (val) of doc similarity (centr):', np.sum(np.abs(sim_mat_theoretical_val - sim_mat_actual_centr_val))/len(doc_topics_val_0))

Difference in evaluation (train) of doc similarity (node 0): 513.9165318187355
Difference in evaluation (val) of doc similarity (node 0): 132.7143677569648
Difference in evaluation (train) of doc similarity (centr): 473.8053300834349
Difference in evaluation (val) of doc similarity (centr): 158.36713751694188


### Topic-words

In [36]:
print('Tópicos (equivalentes) evaluados correctamente (node 0):', np.sum(np.max(np.sqrt(word_topic).dot(np.sqrt(topic_vectors.T)), axis=0)))
print('Tópicos (equivalentes) evaluados correctamente (centr):', np.sum(np.max(np.sqrt(word_topic_centr).dot(np.sqrt(topic_vectors.T)), axis=0)))

Tópicos (equivalentes) evaluados correctamente (node 0): 5.184709469167052
Tópicos (equivalentes) evaluados correctamente (centr): 6.691780624053224
